# 좋은 코드의 일반적인 특징

- 견고한 소프트웨어의 개념 이해
- 작업 중 잘못된 데이터 다루는 법
- 새로운 요구 사항을 쉽게 받아들이고 확장할 수 있는 유지보수가 쉬운 소프트웨어 설계
- 재사용 가능한 소프트웨어 설계
- 개발팀의 생산성을 높이는 효율적인 코드 작성

### 조용한 에러 처리를 피하자!

## DbC (Design by Contract)

- Contract: 소프트웨어 컴포넌트간의 통신 중에 반드시 지켜져야 할 규칙을 문서화한 것
    - 사전 조건: 코드가 실행되기 진행되는 **유횽성 검사**, 사전 조건이 충족되지 않으면 코드를 실행하지 않음
    - 사후 조건: 코드의 실행 결과가 기대한 것인지 확인
    - 불변식: 함수가 실행되는 동안 일정하게 유지되는 것, 함수의 로직에 문제가 없는지 확인하기 위한 것
    - 부작용: 예상되는 코드의 부작용 
- 사전 조건과 사후 조건을 구분하는 것 만으로, 에러가 발생했을 때 어디를 수정해야 하는 지 알 수 있음
    - 사전 조건과 사후 조건의 유효성 검사는 **Decorator**를 사용하는 것을 고려할 수 있음
    - 사전 조건 미충족: 클라이언트 잘못
    - 사후 조건 미충족: 컴포넌트 잘못
- 핵심 구현 외에 계약에 대한 추가 구현이 발생하지만, 장기적인 품질 확보가 가능함
    - 필요한 부분을 잘 정해서 유효성 검사 진행

## Defensive programming

- DbC와는 다른 접근법 (상호보완적인 관계)
- 객체, 함수 또는 메서드와 같은 코드의 모든 부분을 유효하지 않은 것으로브로 스스로 보호할 수 있게 함
- 예상할 수 있는 시나리오의 오류 처리
    - 에러 핸들링 프로시져: 예상되는 에러를 스스로 해결하고 이어서 진행할 지, 프로그램을 중단할 지 결정
        - 값 대체: 예상한 결과 값이 아니면 default 값으로 대체
        - 에러 로깅
        - *예외 처리*: 예외는 캡슐화를 약화시키므로, 예외가 많이 발생하는 함소는 더 작은 단위로 나누는 것이 좋음
- 발생하지 않아야 하는 오류 처리
    - assertion
        - 잘못된 시나리오에 도달할 경우 프로그램을 보호하기 위해서 **무조건 중지**

### 올바른 수준의 추상화 단계에서 예외 처리

- 예외는 해당 예외를 발생/처리하는 함수의 한 부분이어야함

In [1]:
class DataTransport:
    retry_threshold: int = 5
    retry_n_times: int = 3
    
    def __init__(self, connector):
        self._connector = connertor
        self.connect = None
    
    def deliver_event(self, event):
        try:
            self.connect()
            data = event.decode()
            self.send(data)
        except ConnectionError as e:
            logger.info("connection failed: %s", e)
            raise
        except ValueError as e:
            logger.error("%r 잘못된 데이터 포함: %s", event, e)
            raise
            
    def connect(self):
        for _ in range(self.retry_n_times):
            try:
                self.connection = self._connector.connect()
            except ConnectionError as e:
                logger.info(
                    "%s: 새로운 연결 시도 %is",
                    e,
                    self.retry_threshold,
                )
                time.sleep(self.retry_threshold)
            else:
                return self.connection
        raise ConnectionError(
            f'{self.retry_n_times} 번째 재시도 연결 실패'
        )
        
    def send(self, data):
        return self.connection.send(data)

- 위 코드의 문제점
    - `connect`에서 던진 에러를 `deliver_event`에서 처리 -> 캡슐화 저하
    - `deliver_event`에서 발생하는 `ConnectionError`와 `ValueError`는 서로 관련이 없음 
    => 각각 관련 있는 함수에서 처리해야함

In [2]:
def connect_with_retry(connector, retry_n_times, retry_threshold=5):
    for _ in range(self.retry_n_times):
        try:
            return connector.connect()
        except ConnectionError as e:
            logger.info(
                "%s: 새로운 연결 시도 %is",
                e,
                self.retry_threshold,
            )
            time.sleep(retry_threshold)
            
    exc = ConnectionError(
        f'{self.retry_n_times} 번째 재시도 연결 실패'
    )
    logger.exception(exc)
    raise exc
        
class DataTransport:
    retry_threshold: int = 5
    retry_n_times: int = 3
    
    def __init__(self, connector):
        self._connector = connertor
        self.connect = None
    
    def deliver_event(self, event):
        self.connection = connect_with_retry(
            self._connector, self.retry_n_times, self.retry_threshold
        )
        self.send(data)
            
        
    def send(self, event):
        try:
            return self.connection.send(event.decode())
        except ValueError as e:
            logger.error("%r 잘못된 데이터 포함: %s", event, e)
            raise

그 외
- Traceback 노출 금지
    - 보안 이슈 (사용자에게 절대 보여서는 안됨)
- 비어있는 except block 지양
    - 너무 방어적이어서 에러가 조용히 전달됨, **The Zen of Python**과 맞지 않음
- customize error를 사용할 때에는 원본 예외를 포함해야 함
    - `raise <customize error> from <origin error>`

## 관심사의 분리

- 책임이 다르면 컴포넌트, 계층 또는 모듈로 분리되어야 함
- 프로그램의 각 부분은 다른 부분에 대해서 알 필요없이 자신의 기능만 잘 수행하도록 함

### cohesion과 coupling

- cohesion(응집력)
    - 객체가 작고 잘 정의된 목적을 가져야 하며 가능하면 작아야 함
    - cohesion이 높을수록 더 유용하고 재사용성이 높아짐
- coupling(결합력)
    - 두 개 이상의 객체가 서로 어떻게 의존하는지를 나타냄
    - 의존성이 너무 높으면 재사용성이 낮아지고, 변화의 파급 효과가 생기고, 낮은 수준의 추상화만 가능해짐

## 개발 지침 약어

- DRY(Don't Repeat Yourself)/OAOO(Once and Only Once)
    - 코드를 변경하려고 할 때 수정이 필요한 곳은 단 한군데만 있어야 함
    - 코드 중복은 오류가 발생하기 쉽고, 비용(시간)이 비싸고, 완결성이 떨어짐
- YAGNI(You Ain't Gonna Need It)
    - 과잉 엔지니어링을 경계하자
    - 디자인을 할 때 내린 결정으로 특별한 제약 없이 개발을 계속할 수 있다면, 굳이 필요없는 추가 개발을 하지 말 것
    - 미래를 과하게 예측하지 말 것
- KIS(Keep It Simple)
    - 과잉 엔지니어링을 경계하자
    - 문제를 올바르게 해결하는 최소한의 기능을 구현하고 필요한 것 이상으로 솔루션을 복잡하게 만들지 말 것
- EAFP(Easier to Ast Forgiveness that Permission)/ LBYL(Look Before You Leap)
    - EAFP는 코드를 실행해보고 생기는 에러를 다루는 것, LBYL은 코드를 실행하기 전에 살피는 것
    - pythondms EAFP를 선호하므로, `if`보다는 `except`을 권장함

> python tip) 하나씩 값을 대입하는 것 보다는, 한번에 주는 것이 simple함 예시) 입력받은 값들을 하나씩 `setattr()`하는 것보단, input_dict를 만든 뒤 `self.__dict__.update(input_dict)`하는 것이 더 좋음

## 컴포지션과 상속

- 상속을 단지 코드의 재사용성만을 위해서 사용하는 것은 좋지 않다 => coupling 증가
- 상속의 좋은 예
    - public method와 속성 인터페이스를 정의한 컴포넌트가 있을 때, 이 클래스의 기능을 그래도 물려받으면서 추가 기능을 더하려는 경우
        - 즉, 물려받는 기능의 대부분을 사용하는게 전제가 되어야함
    - 인터페이스 정의
        - 기본 추상 클래스를 만든 뒤, 하위 클래스에서 적절한 구현을 하도록 강제
- 상속을 올바르게 하면 객체를 전문화하고 기본 객체에서 출발하여 세부적인 추상화 가능
- 다이아몬드 문제: 4개의 클래스가 다이아몬드 모양으로 다중 상속이 구현된 상황에서, 손자 클래스 입장에서 두 부모 클래스의 메서드 중 어느 것을 사용할 지 모호해지는 문제
    - python은 mro라는 알고리즘을 통해서 같은 부모 클래스 내에서도 순서가 생김. 이는 {class_name}.mro()로 알 수 있음
- mixin
    - 코드를 재사용하기 위해 일반적인 행동을 캡슐화해놓은 기본 클래스
    - 보통은 다른 클래스와 함께 믹스인 클래스를 다중 상속해서 믹스인에 있는 메서드나 속성 사용

In [17]:
# mixin example

class BaseTokenizer:
    def __init__(self, str_token):
        self.str_token = str_token
        
    def __iter__(self):
        yield from self.str_token.split("-")
        
class UpperIterableMixin:
    def __iter__(self):
        return map(str.upper, super().__iter__())
    
class Tokenizer(UpperIterableMixin, BaseTokenizer):
    pass

In [18]:
tk = Tokenizer("sung-jun, 010-9586-2258")

In [19]:
list(tk)

['SUNG', 'JUN, 010', '9586', '2258']

In [20]:
Tokenizer.mro()

[__main__.Tokenizer,
 __main__.UpperIterableMixin,
 __main__.BaseTokenizer,
 object]

## 변수 관련 tip

- python은 call by value로, 값을 복사해서 전달한다.
    - 따라서 가변 인자를 넘겨줄 때는 주의해야한다.
- 인자의 개수가 많으면 좋지않고, 문서화를 잘 할 필요가 있다.
- `*args, **kargs`는 권장하지 않는다.

In [21]:
# packing 연습
def show(e, rest):
    print(f"first: {e}, rest: {rest}")

In [22]:
first, *rest = [1, 2, 3, 4, 5]
show(first, rest)

first: 1, rest: [2, 3, 4, 5]


In [24]:
*rest, last = [1, 2, 3, 4, 5]
show(last, rest)

first: 5, rest: [1, 2, 3, 4]


In [26]:
first, *middle, last = [1, 2, 3, 4, 5]
print(first)
print(middle)
print(last)

1
[2, 3, 4]
5


In [27]:
first, last, *empty = [1, 2]
print(first)
print(last)
print(empty)

1
2
[]
